In [2]:
import ee
ee.Initialize()

import import_ipynb
import NCEP_TWP as NCEP_TPW
import SMWalgorithm as LST
import Compute_emmisibity as EM
import Compute_FVC as FVC
import Compute_NDVI as NDVI
import cloudmask as cloudmask



COLLECTION = ee.Dictionary({
  'L4': {
    'TOA': ee.ImageCollection('LANDSAT/LT04/C01/T1_TOA'),
    'SR': ee.ImageCollection('LANDSAT/LT04/C01/T1_SR'),
    'TIR': ['B6',]
  },
  'L5': {
    'TOA': ee.ImageCollection('LANDSAT/LT05/C01/T1_TOA'),
    'SR': ee.ImageCollection('LANDSAT/LT05/C01/T1_SR'),
    'TIR': ['B6',]
  },
  'L7': {
    'TOA': ee.ImageCollection('LANDSAT/LE07/C01/T1_TOA'),
    'SR': ee.ImageCollection('LANDSAT/LE07/C01/T1_SR'),
    'TIR': ['B6_VCID_1','B6_VCID_2'],
  },
  'L8': {
    'TOA': ee.ImageCollection('LANDSAT/LC08/C01/T1_TOA'),
    'SR': ee.ImageCollection('LANDSAT/LC08/C01/T1_SR'),
    'TIR': ['B10','B11']
  }
})

def collection(landsat, date_start, date_end, geometry, use_ndvi):
    
    collection_dict = ee.Dictionary(COLLECTION.get(landsat))
    
    
    landsatTOA = ee.ImageCollection(collection_dict.get('TOA')) \
                .filter(ee.Filter.date(date_start, date_end)) \
                .filterBounds(geometry) \
                .map(cloudmask.toa)

      # load Surface Reflectance collection for NDVI
    landsatSR = ee.ImageCollection(collection_dict.get('SR')) \
                .filter(ee.Filter.date(date_start, date_end)) \
                .filterBounds(geometry) \
                .map(cloudmask.sr) \
                .map(NDVI.addBand(landsat)) \
                .map(FVC.addBand(landsat)) \
                .map(NCEP_TPW.addBand) \
                .map(EM.addBand(landsat,use_ndvi))

      # combine collections
      # all channels from surface reflectance collection
      # except tir channels: from TOA collection
      # select TIR bands
    tir = ee.List(collection_dict.get('TIR'))
    landsatALL = (landsatSR.combine(landsatTOA.select(tir), True))

      # compute the LST
    landsatLST = landsatALL.map(LST.addBand(landsat))
    
    
    return landsatLST
